In [90]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import  torchvision.transforms as transforms
from torchvision import datasets, transforms,models
from torch.utils.data import DataLoader
from torchvision.models import GoogLeNet_Weights


In [91]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [92]:
train_dir = '/content/drive/MyDrive/Computer Vision/flowers/train'
eval_dir = '/content/drive/MyDrive/Computer Vision/flowers/val'

In [93]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
}

train_dataset = datasets.ImageFolder(train_dir,transform=data_transforms['train'])
eval_dataset = datasets.ImageFolder(eval_dir,transform=data_transforms['val'])

train_loader = DataLoader(train_dataset,batch_size = 16,shuffle=True)
eval_loader = DataLoader(eval_dataset,batch_size = 16)

In [94]:
model = models.googlenet(pretrained = True)
print(model)

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [95]:
from torch._C import device
model.fc = nn.Linear(in_features=1024,out_features=5)
print(model)
for param in model.parameters():
  param.requires_grad = False
for param in model.fc.parameters():
  param.requires_grad = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [96]:
total_params = sum(p.numel() for p in model.parameters())
print(total_params)

5605029


In [97]:
critertion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = 0.001)

In [98]:
def train_model(model, optimizer, train_loader, eval_loader, epochs=5):
    for epoch in range(epochs):
        model.train()
        train_correct = 0
        train_total = 0
        running_loss = 0.0

        print(f"\nEpoch {epoch+1}/{epochs}")

        for i, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = critertion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            train_correct += (preds == labels).sum().item()
            train_total += labels.size(0)

            if (i + 1) % 10 == 0:
                batch_acc = (preds == labels).float().mean().item()
                print(f"[Batch {i+1}/{len(train_loader)}] "
                      f"Loss: {loss.item():.4f}, Batch Acc: {batch_acc:.4f}")


        train_acc = train_correct / train_total
        print(f"Epoch {epoch+1} Summary - "
              f"Loss: {running_loss:.4f}, "
              f"Train Accuracy: {train_acc:.4f}")


        model.eval()
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for images, labels in eval_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)

        val_acc = val_correct / val_total
        print(f"Validation Accuracy: {val_acc:.4f}")


In [99]:
# train_model(model,optimizer,train_loader,eval_loader)

In [100]:
class ConvBlock(nn.Module):
  def __init__(self,in_channel,out_channel,kernel_size,stride=1,padding=0):
     super().__init__()
     self.conv = nn.Conv2d(
         in_channels=in_channel,
         out_channels=out_channel,
         kernel_size=kernel_size,
         padding=padding,
         bias = False
     )
     self.relu = nn.ReLU(inplace=True) # Inplce is true means no new memory is allocated the original tensor is modified

  def forward(self,x):
    x = self.conv(x)
    x = self.relu(x)

    return x

In [101]:
class InceptionModule(nn.Module):
  def __init__(self,
               in_channel,
               ch1x1,
               ch3x3red,ch3x3,
               ch5x5red,ch5x5,
               pool_proj
               ):
     super().__init__()

     self.branch1 = ConvBlock(in_channel,ch1x1,kernel_size=1)

     self.branch2 = nn.Sequential(
         ConvBlock(in_channel,ch3x3red,kernel_size=1),
         ConvBlock(ch3x3red,ch3x3,kernel_size=3,padding = 1)
     )

     self.branch3 = nn.Sequential(
         ConvBlock(in_channel,ch5x5red,kernel_size=1),
         ConvBlock(ch5x5red,ch5x5,kernel_size=5,padding = 2)
     )

     self.branch4 = nn.Sequential(
        nn.MaxPool2d(kernel_size=3,stride=1,padding = 1),
        ConvBlock(in_channel,pool_proj,kernel_size=1)
    )

  def forward(self,x):
    return torch.concat([
        self.branch1(x),
        self.branch2(x),
        self.branch3(x),
        self.branch4(x),
    ],dim=1)




In [102]:
class GoogLeNet(nn.Module):
    def __init__(self, num_classes=1000):
        super().__init__()

        self.stem = nn.Sequential(
            ConvBlock(3, 64, kernel_size=7, stride=2, padding=3),
            nn.MaxPool2d(3, stride=2, padding=1),
            ConvBlock(64, 64, kernel_size=1),
            ConvBlock(64, 192, kernel_size=3, padding=1),
            nn.MaxPool2d(3, stride=2, padding=1),
        )

        self.inception3a = InceptionModule(192, 64, 96, 128, 16, 32, 32)
        self.inception3b = InceptionModule(256, 128, 128, 192, 32, 96, 64)
        self.maxpool = nn.MaxPool2d(3, stride=2, padding=1)

        self.inception4a = InceptionModule(480, 192, 96, 208, 16, 48, 64)
        self.inception4b = InceptionModule(512, 160, 112, 224, 24, 64, 64)
        self.inception4c = InceptionModule(512, 128, 128, 256, 24, 64, 64)
        self.inception4d = InceptionModule(512, 112, 144, 288, 32, 64, 64)
        self.inception4e = InceptionModule(528, 256, 160, 320, 32, 128, 128)

        self.inception5a = InceptionModule(832, 256, 160, 320, 32, 128, 128)
        self.inception5b = InceptionModule(832, 384, 192, 384, 48, 128, 128)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(0.4)
        self.fc = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = self.stem(x)

        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.maxpool(x)

        x = self.inception4a(x)
        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        x = self.inception4e(x)
        x = self.maxpool(x)

        x = self.inception5a(x)
        x = self.inception5b(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc(x)

        return x


In [103]:
model = GoogLeNet(1000)
print(model)

GoogLeNet(
  (stem): Sequential(
    (0): ConvBlock(
      (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
      (relu): ReLU(inplace=True)
    )
    (1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (2): ConvBlock(
      (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (relu): ReLU(inplace=True)
    )
    (3): ConvBlock(
      (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (relu): ReLU(inplace=True)
    )
    (4): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (inception3a): InceptionModule(
    (branch1): ConvBlock(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (relu): ReLU(inplace=True)
    )
    (branch2): Sequential(
      (0): ConvBlock(
        (conv): Conv2d(192, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (relu): ReLU(inplace=True)
      )
      (1): 

In [104]:
total_params = sum(p.numel() for p in model.parameters())
print(total_params)

6991272
